In [234]:
from panel.interact import interact
import math

from pathlib import Path
from scipy.stats import levy_stable, wrapcauchy
from scipy.spatial import distance
from typing import Optional
import numpy as np
import pandas as pd
import param
import plotly.graph_objects as go
import panel as pn
import panel.widgets as pnw
import holoviews as hv
pn.extension("plotly", sizing_mode="stretch_width")

radio_group_trajectory = pn.widgets.RadioButtonGroup(name="Trajectory", options=["BM", "CRW", "LF"],
button_type="default")
n_steps = pnw.IntSlider(name="n_steps", width=100, value=30, step=1, start=30, end=100000)
metric = pnw.Select(name="metric", options=["PL", "MSD"])
x_pos = pnw.IntInput(name="x_pos", value=0, start=0, end=10000, step=1)
y_pos = pnw.IntInput(name="y_pos", value=0, start=0, end=10000, step=1)
speed = pnw.IntSlider(name="speed", width=50, value=1, step=1, start=1, end=1000)
crw_exponent = pnw.FloatInput(name="c exponent", value=0.0, start=0.1, end=1, step=0.1)
alpha = pnw.FloatInput(name="alpha", value=0.0, start=0.1, end=2, step=0.1)
beta = pnw.FloatInput(name="beta", value=0.0, start=0.1, end=1, step=0.1)
location = pnw.IntSlider(name="loc", width=50, value=0, step=1, start=0, end=10)

# Trajectory data
trajectory_df = None

class Vec2d(object):
    """
    2d vector class, supports vector and scalar operators, and also provides a bunch of high level functions
    http://www.pygame.org/wiki/2DVectorClass
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)
    
def single_euclidean_distance(pointA: pd.core.series.Series, pointB: pd.core.series.Series) -> float:
    """_summary_

    :param pointA: _description_
    :type pointA: pd.core.series.Series
    :param pointB: _description_
    :type pointB: pd.core.series.Series
    :return: _description_
    :rtype: float
    """
    ed = math.sqrt(math.pow((pointB[0] - pointA[0]), 2) + math.pow((pointB[1] - pointA[1]), 2))
    return ed

def brownian_motion(n_steps: int = 1000, speed: int = 6, x_pos: int= 0, y_pos = 0) -> pd.DataFrame:
    """_summary_

    :param n_steps: _description_, defaults to 1000
    :type n_steps: int, optional
    :param speed: _description_, defaults to 6
    :type speed: int, optional
    :param s_pos: _description_, defaults to [0, 0]
    :type s_pos: list, optional
    :return: _description_
    :rtype: pd.DataFrame
    """
    #print(s_pos[0])
    # Init velocity
    velocity = Vec2d(x_or_pair=speed, y=0)
    # Init dataframe
    BM_2d_df = pd.DataFrame(columns=["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": x_pos, "y_pos": y_pos}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)  # TYPE OF DISTRIBUTION
        velocity = velocity.rotated(turn_angle)
        temp_df = pd.DataFrame([{"x_pos": BM_2d_df.x_pos[i]+velocity.x, "y_pos":  BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    global trajectory_df
    trajectory_df = BM_2d_df
    #print(<c.shape[0])
    return plot_figure(trace=BM_2d_df)

def calculate_mean_sqrt_displacement() -> pd.DataFrame:
    """_summary_

    :param dataframe: _description_
    :type dataframe: pd.DataFrame
    :return: _description_
    :rtype: pd.DataFrame
    """
    # Empty MSD
    #N = trajectory_df.shape[0]
    N = 100
    dataframe = trajectory_df
    MSD = np.zeros([N])
    cum_sum = 0

    for tau in range(1, N):
        for i in range(1, N-tau+1):
            cum_sum += math.pow(single_euclidean_distance(dataframe.iloc[i+tau-1], dataframe.iloc[i-1]), 2)
        MSD[tau] = (cum_sum/(N-tau))
        cum_sum = 0
    return plot_figure(trace=MSD, plot_type="lineal")

def plot_figure(trace: pd.DataFrame, plot_type: str="trajectory", dimension: str = "2d"):
    figure = go.Figure()

    if plot_type == "trajectory":
        figure.add_trace(go.Scatter3d(x=trace.x_pos,
                                      y=trace.y_pos,
                                      z=trace.index,
                                      marker=dict(size=2),
                                      line=dict(color="blue", width=2),
                                      mode="lines",
                                      name="BM_2D",
                                      showlegend=True))
    else:
        figure.add_trace(go.Scatter(
            x=trajectory_df.index,
            y=trace,
            marker=dict(size=2),
            line=dict(width=2),
            mode="lines",
            name="metric",
            showlegend=True
        ))
        

    #figure.show()
    return figure

@pn.depends(radio_group_trajectory)
def create_options(trajectory):
    if trajectory == "BM":
        return pn.Column(pn.Row(n_steps), pn.Row(x_pos, y_pos), pn.Row(speed), pn.Row(metric))
    if trajectory == "CRW":
        return pn.Column(pn.Row(n_steps), pn.Row(x_pos, y_pos), pn.Row(speed), pn.Row(crw_exponent), pn.Row(metric))
    elif trajectory == "LF":
        return pn.Column(pn.Row(n_steps), pn.Row(x_pos, y_pos), pn.Row(speed), pn.Row(alpha, beta), pn.Row(location), pn.Row(metric))

@pn.depends(n_steps, x_pos, y_pos, speed, crw_exponent, alpha, beta, location)
def trajectories(n_steps, x_pos, y_pos, speed, crw_exponent, alpha, beta, location):
    if radio_group_trajectory.value=="BM":
        figure = brownian_motion(n_steps, speed, x_pos, y_pos)
        return figure
        
    #print("trajectoria")
    #if radio_group_trajectory.value=="BM":
        #trajectory = RW_class.brownian_motion(n_steps, speed, list(x_pos, y_pos))
        #print("entro")

@pn.depends(metric)
#@interact(metric_value=trajectory_df)
def metrics(metric_value):
    if metric_value=="MSD":
        #print(trajectory_df)
        figure = calculate_mean_sqrt_displacement()
        return figure

layout = interact(create_options, trajectory=radio_group_trajectory, throttled=True)
#layout_metrics =interact(metrics, metric_value=metric)
pn.Column(pn.Row(layout, trajectories, metrics))


Traceback (most recent call last):
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\pyviz_comms\__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\viewable.py", line 292, in _on_msg
 doc.unhold()
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\bokeh\document\document.py", line 799, in unhold
 self.callbacks.unhold()
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\bokeh\document\callbacks.py", line 396, in unhold
 self.trigger_on_change(event)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\bokeh\document\callbacks.py", line 373, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\bokeh\util\callback_manager.py", line 191, in invoke
 callback(attr, old, new)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\reactive.py", line 392, in _comm_change
 self._schedule_change(doc, comm)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\reactive.py", line 376, in _schedule_change
 self._change_event(doc)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\reactive.py", line 370, in _change_event
 self._process_events(events)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\reactive.py", line 315, in _process_events
 self.param.update(**self_events)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\param\parameterized.py", line 1898, in update
 self_._batch_call_watchers()
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\param\parameterized.py", line 2059, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\param\parameterized.py", line 2021, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\param.py", line 794, in _replace_pane
 new_object = self.eval(self.object)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\panel\param.py", line 777, in eval
 return function(*args, **kwargs)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\param\parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "C:\Users\njuarez\AppData\Local\Temp\ipykernel_17736\1291058166.py", line 196, in metrics
 figure = calculate_mean_sqrt_displacement()
 File "C:\Users\njuarez\AppData\Local\Temp\ipykernel_17736\1291058166.py", line 139, in calculate_mean_sqrt_displacement
 cum_sum += math.pow(single_euclidean_distance(dataframe.iloc[i+tau-1], dataframe.iloc[i-1]), 2)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\pandas\core\indexing.py", line 1074, in __getitem__
 return self._getitem_axis(maybe_callable, axis=axis)
 File "C:\Users\njuarez\Documents\Documents\06_Personal\UDG\1er_semestre\Topico_i\Git\venv\lib\site-packages\pandas\core\indexing.py", line 1626, in _getitem_axis
 self._validate_integer(key, axi

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] Column(sizing_mode='stretch_width')
            [0] Column(sizing_mode='stretch_width')
                [0] RadioButtonGroup(name='Trajectory', options=['BM', 'CRW', 'LF'], sizing_mode='stretch_width', value='BM')
            [1] Row(sizing_mode='stretch_width')
                [0] Column(sizing_mode='stretch_width')
                    [0] Row(sizing_mode='stretch_width')
                        [0] IntSlider(end=100000, name='n_steps', sizing_mode='stretch_width', start=30, value=30, width=100)
                    [1] Row(sizing_mode='stretch_width')
                        [0] IntInput(end=10000, name='x_pos', sizing_mode='stretch_width', start=0)
                        [1] IntInput(end=10000, name='y_pos', sizing_mode='stretch_width', start=0)
                    [2] Row(sizing_mode='stretch_width')
                        [0] IntSlider(end=1000, name='speed', sizing_mode='stretch_width', start=1, value=1, width=50)
                    [3] Row(sizing_mode='stretch_width')
                        [0] Select(name='metric', options=['PL', 'MSD'], sizing_mode='stretch_width', value='PL')
        [1] ParamFunction(function, sizing_mode='stretch_width')
        [2] ParamFunction(function, sizing_mode='stretch_width')